In [0]:
# Setup de Importação 
import sys
import os

# Pega o caminho do notebook atual
caminho_notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
caminho_workspace = f"/Workspace{caminho_notebook}"

# Sobe dois níveis para achar a Raiz
pasta_notebooks = os.path.dirname(caminho_workspace)
raiz_projeto = os.path.dirname(pasta_notebooks)

#. Adiciona ao Path
sys.path.append(raiz_projeto)
print(f"Raiz do projeto adicionada: {raiz_projeto}")

In [0]:
# Importação e Configuração Dinâmica de Path
from pyspark.sql.functions import col, lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType
import time
from delta.tables import *
from src.utils.ibge_api import buscar_dados_sidra
from src.utils.delta_utils import salvar_delta_merge

In [0]:
# Definição dos Widgets
dbutils.widgets.text("camada", "bronze", "1. Camada (Schema)")
dbutils.widgets.text("nome_tabela", "api_ibge_6579", "2. Nome da Tabela (Sem schema)")

# Captura dos valores
p_camada = dbutils.widgets.get("camada")
p_tabela = dbutils.widgets.get("nome_tabela")

In [0]:
# Definição Explícita do Schema (Evita Job Spark custoso de inferência), além disso garantimos que caso haja alguma mudança na estrutura da API o processo não irá quebrar. E caso a área veja necessidade de alterar para essa nova mudança, basta alterar o schema e rodar novamente o processo.
colunas_ibge = [
    "NC", "NN", "MC", "MN", "V", 
    "D1C", "D1N", "D2C", "D2N", "D3C", "D3N"
]

# Garantir o tipo tudo string para evitar erros
fields = [StructField(c, StringType(), True) for c in colunas_ibge]
sidra_schema = StructType(fields)

In [0]:
# Criação da lista para armazenar os dados
lista_dados_completa = []

# URL da API do IBGE para dados demográficos
url_macro = "https://apisidra.ibge.gov.br/values/t/6579/n1/all/n2/all/n3/all/p/all/v/all"

# Chamada para obter os dados macro (Brasil, Regiões, Estados)
dados_macro = buscar_dados_sidra(url_macro, retries = 3)

# Adicionar os dados macro à lista
lista_dados_completa.extend(dados_macro)

print('Dados macro obtidos:', len(lista_dados_completa))

In [0]:
# Extração Micro (Municípios  por Estado)
codigos_uf = [
    11, 12, 13, 14, 15, 16, 17, # Norte
    21, 22, 23, 24, 25, 26, 27, 28, 29, # Nordeste
    31, 32, 33, 35, # Sudeste
    41, 42, 43, # Sul
    50, 51, 52, 53 # Centro-Oeste e DF
]

print(f"Iniciando loop de extração de Municípios por {len(codigos_uf)} estados...")

for uf in codigos_uf:
    # A sintaxe "n6/in n3 {uf}" pede os municípios DENTRO daquele estado específico
    url_municipios = f"https://apisidra.ibge.gov.br/values/t/6579/n6/in%20n3%20{uf}/p/all/v/all"
    
    # Chamada via utils
    dados_uf = buscar_dados_sidra(url_municipios, retries=3, backoff_factor=2)
    
    if dados_uf:
        lista_dados_completa.extend(dados_uf)
        print(f"  -> UF {uf}: OK ({len(dados_uf)} registros)")
    else:
        # Se retornar vazio pra alguma UF o processo para
        raise Exception(f"Erro ao obter dados para UF {uf}.")
    
    # Pausa de cortesia para não tomar Block 429 da API
    time.sleep(0.5)

print(f"\n Extração Concluída. Total de registros em memória: {len(lista_dados_completa)}")

In [0]:
# Conversão da lista para dfs
dfs_raw = spark.createDataFrame(lista_dados_completa, schema=sidra_schema)

# Removemos o cabeçalho técnico que vem no JSON (onde NC = "Nível Territorial...") Como a extração é em Loop a opção por fazer a eliminação do cabeçalho técnico é uma boa prática e garante que foi eliminado em todos os loops

dfs_staging = dfs_raw.filter(col("NC") != "Nível Territorial (Código)") \
    .withColumn("data_ingestao_bronze", current_timestamp()) \
    .withColumn("origem", lit("API_SIDRA_6579"))

In [0]:
# Cria o Schema se não existir
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {p_camada}")

# Define o caminho final
tabela_destino_full = f"{p_camada}.{p_tabela}"

print(f"Destino final: {tabela_destino_full}")

In [0]:
# Define as chaves para a Bronze 
chaves_bronze = [
    "NC", "NN", "MC", "MN", "V", 
    "D1C", "D1N", "D2C", "D2N", "D3C", "D3N"
]

In [0]:
# Escrita da tabela usando função de escrita da utils
salvar_delta_merge(
    df=dfs_staging,
    tabela_destino=tabela_destino_full,
    chaves_match=chaves_bronze,
    modo_carga_inicial="overwrite",
    fazer_update=False 
)

In [0]:
%sql
select * from bronze.api_ibge_6579